Check for matches for product molecule in generated reaction network

In [1]:
from rdkit import Chem
from rdkit.DataStructs import FingerprintSimilarity as tanimoto
from rdkit.DataStructs.cDataStructs import ExplicitBitVect
from rdkit.Chem import Draw
from rdkit.Chem import AllChem,MACCSkeys
import pandas as pd
from ast import literal_eval
from joblib import Parallel, delayed

In [2]:
def GenMACCS(smiles):
    #smiles is a list of smiles
    fp=[]
    for x in smiles:
        if not(x.isdigit()):
            fp.append(MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles(x)))
        else:
            fp.append(ExplicitBitVect(167))
    
    return fp

In [3]:
def Similarity(smiles,target):
    score=[]
    for x in smiles:
        s=0
        if not (x.isdigit()):
            try:
                mol=Chem.MolFromSmiles(x)
                for t in target:
                    if  mol.HasSubstructMatch(t):
                        s+=1
                    else:
                        s+=0
                score.append(s)
            except:
                score.append(-1)
    return score

In [66]:
df=pd.read_csv('Biomass/HMFRev_NoRepeat_Reduced_Nghb.csv',header=0)
keyslist=df.keys()[1:]

C:\Users\ks\AppData\Local\Temp\ipykernel_6736\1273860479.py:1: DtypeWarning: Columns (1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,111,112,113,114,115,116,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,182,183,184,185,186,187,188,189,190,191,192,193,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,234,235,236,237,238,239,240,241,242,243,244,245,246,248,249,250,251,252,253,254,255,256,257,258,259,260,262,263,264,265,26

In [83]:
targetfp=MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles('OCc1ccco1'))
#targetfp=MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles('C1C2C(C(C(C(O1)O2)O)O)O'))
#targetmol=Chem.MolFromSmarts('C1=CC=C(C=C1)CCOC2=CC=CC=C2')

In [ ]:
#alkane=Chem.MolFromSmarts('[CX4]')
#alkene=Chem.MolFromSmarts('[$([CX3]=[CX3])]')
#alkyne=Chem.MolFromSmarts('[$([CX2]#[CX2])]')
#arene=Chem.MolFromSmarts('[c]')
#ketone=Chem.MolFromSmarts('[#6][CX3](=[O])[#6]') 
#aldehyde=Chem.MolFromSmarts('[CX3H1](=[O])')
#ester=Chem.MolFromSmarts('[#6][CX3](=[O])[OX2H0][#6]')
#acid=Chem.MolFromSmarts('[#6][CX3](=[O])[OX2H1]')
#alcohol=Chem.MolFromSmarts('[CHX4][OX2H]')
#amine=Chem.MolFromSmarts('[NX3;H2,H1;!$(NC=O)]')
#halide=Chem.MolFromSmarts('[CX4][F,Cl,Br,I]')
#ether=Chem.MolFromSmarts('[OD2]([#6])[#6]')
#nitro=Chem.MolFromSmarts('[$([NX3](=O)=O),$([NX3+](=O)[O-])][!#8]')
#alring=Chem.MolFromSmarts('[$([CX4;R])]')


In [79]:
smarts=['[c]','[$([CX3]=[CX3])]']
targetmol=[Chem.MolFromSmarts(s) for s in smarts]
#Chem.SanitizeMol(targetmol)
#targetfp=MACCSkeys.GenMACCSKeys(targetmol)


In [84]:
similarity=pd.DataFrame(columns=['Smiles','Gen','Id','Similarity'])
for key in  df.keys()[1:]:
    smileslist=df[key]
    smileslist=smileslist.dropna()
    newlist=[]
    for s in smileslist:
        newlist.append(literal_eval(s))
    FingerPrints=Parallel(n_jobs=-1,verbose=1)(delayed(GenMACCS)(smiles) for smiles in newlist) #FingerPrints is a list of list of Fingerprints
    for i,fp in enumerate(FingerPrints):
        score=[tanimoto(x,targetfp) for x in fp]
        if any(val>=0.2 for val in score):
            similarity=pd.concat([similarity,pd.DataFrame({'Smiles':str(newlist[i]),'Gen': key,'Id':i,'Similarity':score})],ignore_index=True)
            

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 133 out of 133 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   0 out of   0 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  88 out of  88 | elapsed:    0.

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  42 out of  65 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 out of  19 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  32 task

[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 145 out of 145 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 439 out of 439 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   0 out of   0 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  53 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend wi

[Parallel(n_jobs=-1)]: Done 449 out of 449 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  13 out of  13 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 out of  18 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Usin

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  97 out of  97 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  32 out of  55 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 out of  18 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  95 out of  95 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   0 out of   0 | 

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  47 out of  47 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Usin

[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 335 out of 335 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 209 out of 209 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  94 out of 117 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 117 out of 117 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.0s
[P

[Parallel(n_jobs=-1)]: Done  34 out of  57 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  57 out of  57 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


In [83]:
similarity_sub=pd.DataFrame(columns=['Smiles','Gen','Id','Similarity'])
for key in  df.keys()[1:]:
    smileslist=df[key]
    smileslist=smileslist.dropna()
    newlist=[]
    for s in smileslist:
        newlist.append(literal_eval(s))
    score=Parallel(n_jobs=-1,verbose=1)(delayed(Similarity)(smiles,targetmol) for smiles in newlist) #FingerPrints is a list of list of Fingerprints
    for i,fp in enumerate(score):
        if any(val>=2 for val in fp):
            similarity_sub=pd.concat([similarity_sub,pd.DataFrame({'Smiles':str(newlist[i]),'Gen': key,'Id':i,'Similarity':score})],ignore_index=True)
                   

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  17 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=

[Parallel(n_jobs=-1)]: Done  16 out of  19 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:    0.0s finished


In [86]:
similarity.to_csv('Biomass/PC2 hits/HMF_LGPC2FuranFPhits.csv')

In [85]:
similarity

,Smiles,Gen,Id,Similarity
0,"['CO.[CH]c1ccc(Cl)cc1', '[CH]=N.[CH]N']",Gen111,7,0.200000
1,"['CO.[CH]c1ccc(Cl)cc1', '[CH]=N.[CH]N']",Gen111,7,0.000000
2,"['[CH]C=[CH]', 'Oc1ccc(Cc2ccccc2)cc1']",Gen111,9,0.000000
3,"['[CH]C=[CH]', 'Oc1ccc(Cc2ccccc2)cc1']",Gen111,9,0.304348
4,['O.[CH2]O'],Gen111,11,0.210526
...,...,...,...,...
1585,"['365', '[CH2]OCC=C']",Gen33481,247,0.000000
1586,"['365', '[CH2]OCC=C']",Gen33481,247,0.238095
1587,['[O-][CH]O'],Gen33581,26,0.210526
1588,['[CH2]CCC=CCCCCCCCCO'],Gen33582,43,0.320000


In [44]:
similarity_sub.to_csv('Biomass/PC2 hits/PC1_PC2Subhits.csv')

In [84]:
similarity_sub

,Smiles,Gen,Id,Similarity
